# Building a RAG pipeline

In [1]:
import os

# Set the LANGCHAIN_TRACING_V2 environment variable to enable tracing for LangChain version 2
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Set the LANGCHAIN_API_KEY environment variable with your LangChain API key
os.environ["LANGCHAIN_API_KEY"] = ""

# Set the OPENAI_API_KEY environment variable with your OpenAI API key
os.environ["OPENAI_API_KEY"] = ""

# Set the PINECONE_API_KEY environment variable with your Pinecone API key
os.environ["PINECONE_API_KEY"] = ""

# Set the LANGCHAIN_PROJECT environment variable to specify the project name for LangChain
os.environ['LANGCHAIN_PROJECT'] = "earnings-call"

# Set the LANGSMITH_USER_HANDLE environment variable with your Langsmith user handle
os.environ['LANGSMITH_USER_HANDLE'] = "bhaskarjit/earning-call-rag"

## Importing the required packages

In [2]:
# Import the json module for handling JSON data
import json

# Import StrOutputParser from langchain_core.output_parsers for parsing string outputs
from langchain_core.output_parsers import StrOutputParser

# Import RunnablePassthrough from langchain_core.runnables for running tasks in a passthrough manner
from langchain_core.runnables import RunnablePassthrough

# Import RunnableParallel from langchain_core.runnables for running tasks in parallel
from langchain_core.runnables import RunnableParallel

# Import ChatPromptTemplate from langchain_core.prompts for creating chat prompt templates
from langchain_core.prompts import ChatPromptTemplate

# Import OpenAIEmbeddings from langchain_openai for working with OpenAI embeddings
from langchain_openai import OpenAIEmbeddings 

# Import ChatOpenAI from langchain_openai for interacting with the OpenAI chat model
from langchain_openai import ChatOpenAI

# Import PineconeVectorStore from langchain_pinecone for managing vector storage in Pinecone
from langchain_pinecone import PineconeVectorStore

### Defining globle variables

In [3]:
# Define the index name for storing or retrieving data, in this case, 'earning-calls'
INDEX_NAME = 'earning-calls'

# Set the number of top results to return, here set to 6
TOP_K = 6

# Specify the quarter for which the data is relevant, in this example, Q1 (Quarter 1)
QUARTER = "Q1"

# Define the filename of the document being processed, in this case, "Adani Enterprises Ltd.pdf"
FILENAME = "Adani Enterprises Ltd.pdf"

# Specify the fiscal year for the data, in this example, FY24 (Fiscal Year 2024)
YEAR = "FY24"

# Initialize OpenAIEmbeddings with the specified model for generating text embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Initialize ChatOpenAI with the specified model and temperature for generating chat responses
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

## Loading Vectorstore

In [4]:
# Create a PineconeVectorStore instance with the specified index name and embedding model
index = PineconeVectorStore(
        index_name = INDEX_NAME,
        embedding = embeddings
    )

# Convert the PineconeVectorStore instance into a retriever object for searching
retriver = index.as_retriever(search_kwargs={"filter":{"quarter": QUARTER, "filename": FILENAME},"k": TOP_K})

In [5]:
retriver.invoke("what is the capex?")

[Document(page_content='mining is, all our commercial mining activities, be it India or overseas will fall under the \ncommercial mining which fall under our natural resource s region of which my colleague Vinay \nis the CEO . \nModerator:  Thank you. The  next question is from the line of Gaurav Singhal from Aspex Management \nLimited . Please go ahead.  \nGaurav Singhal:  Two questions from me. So, one is, can you help give a break up of the $3.7 billion CAPEX plan \nfor this year  across the different segment s? Thank you.  \nRobbie Singh : Approximately about $300 million for Green Hydrogen, $1.1 billi on for airports, approximately \n$1.7 billion for the road network, just under $100 million  for water , just under $200 million for \nthe Data Center, and then small completion cost for the copper project just under $200 million.  \nGaurav Singhal:  And then secondly in terms of financing the CAPEX, the board had passed resolution to al low \nthe group  to raise about 12,500 crores 

In [7]:
# from langchain import hub
# prompt = hub.pull("pallaanji/patient-review")
# print(prompt)

## Creating a prompt template

In [8]:
# This template is designed for a chatbot that acts as an expert Q&A system
chat_template = ChatPromptTemplate.from_messages(
    [
        # Define the system message that outlines the chatbot's role and rules
        ("system", "You are financial advisor who response to the certain Quetion and Answers. Limit your response to the infromation from documents"),
        
        # Define the human message that provides context information and a query
        ("human", "Context information is below.\n---------------------\n{context}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query}\nAnswer: "),
    ]
)


In [9]:
print(chat_template.format(context="THIS IS A SAMPLE CONTEXT TO SEE HOW THE PROMPT LOOKS", query="THIS IS A SAMPLE QUERY?"))

System: You are financial advisor who response to the certain Quetion and Answers. Limit your response to the infromation from documents
Human: Context information is below.
---------------------
THIS IS A SAMPLE CONTEXT TO SEE HOW THE PROMPT LOOKS
---------------------
Given the context information and not prior knowledge, answer the query.
Query: THIS IS A SAMPLE QUERY?
Answer: 


## Creating the RAG chain

In [11]:
# Define a function to format documents by joining their page content with newlines
def format_docs(docs):
    # Join the page content of each document with two newlines between them
    return "\n\n".join(doc.page_content for doc in docs)

# Create a runnable chain for generating responses from formatted documents
# This chain starts with formatting the documents, then uses the chat template,
# processes the response through the language model, and finally parses the output
rag_chain_from_docs = (
    RunnablePassthrough.assign(context = (lambda x: format_docs(x['context'])))
    |chat_template
    |llm
    |StrOutputParser()
)

# Create a runnable chain for retrieving context and generating answers in parallel
# This chain retrieves context using the retriever and formats the query,
# then generates answers using the previously defined chain for formatted documents
rag_chain_with_source = RunnableParallel(
    {"context": retriver, "query": RunnablePassthrough()}
).assign(answer = rag_chain_from_docs)

In [12]:
response = rag_chain_with_source.invoke("What was the income?")
print(response['answer'])

The consolidated total income for the quarter was Rs. 25,810 crores.


## Prompt Versioning

Loading a Specific Version of a Prompt:

1. **Version Tracking in Repositories:**
   - Each push to a prompt repository saves a new version, identified by a unique commit hash.

2. **Loading the Latest Version:**
   - By default, accessing the repo will load the most recent version of a given prompt.

3. **Loading a Specific Version:**
   - To load a specific version, include its commit hash with the prompt name.
   - Example: For loading the "earnings-call-rag" with version `6214c98a`, append this hash to the prompt name in your loading command.

In [ ]:
# !pip install langchainhub

In [27]:
from langchain import hub
prompt = hub.pull("bhaskarjit/earnings-call-rag")

In [28]:
print(prompt.format(context="THIS IS A SAMPLE CONTEXT TO SEE HOW THE PROMPT LOOKS", query="THIS IS A SAMPLE QUERY?"))

System: You are an expert Q&A system that is trusted around the world.
Always answer the query using the provided context information, and not prior knowledge.
Some rules to follow:
1. Never directly reference the given context in your answer.
2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines.
Human: Context information is below.
---------------------
THIS IS A SAMPLE CONTEXT TO SEE HOW THE PROMPT LOOKS
---------------------
Given the context information and not prior knowledge, answer the query.
Use Bullet poits whenever possible in the answer.
Query: THIS IS A SAMPLE QUERY?
Answer: 


In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt # placing the newly loaded prompt here
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriver, "query": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [16]:
response = rag_chain_with_source.invoke("What was the income?")
print(response['answer'])

- The consolidated total income was at Rs. 25,810 crores.


### How to Share Prompts on LangChain Hub:

1. **Getting Started:**
   - Getting prompts from LangChain Hub is easy, and so is sharing your own prompts.

   - This lets you easily share and manage your own prompts.

2. **Making a Prompt:**

   - First, create a prompt that fits what you need.

   - Make sure it follows the rules of LangChain Hub.

3. **Sharing Process:**

   - The sharing has two important parts:
     - **Account Handle:** Your special name in LangChain Hub, like `me-langchain-user`.
     - **Prompt Name:** A clear name for your prompt, showing what it does.

4. **How to Share with Code:**
   - This is a simple way to share:
     ```python
     from langchain import hub

     # Define your prompt
     my_prompt = "..."  # Your prompt content goes here

     # Share it on LangChain Hub
     hub.push(f"{account_handle}/{prompt_name}", my_prompt)
     ```
     - Replace `account_handle` with your username and `my_prompt` with your prompt's name.
     
     - Make sure `my_prompt` follows LangChain PromptTemplate.

5. **Using Your Shared Prompt:**
   - Once it's shared, you can use it in different apps through LangChain Hub.
   - This makes it easy to share with others, work together, and manage your prompts.

In [21]:
len(prompt.messages), prompt.messages

(2,
 [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['\n"patient_name"'], template='You are an assistant that supports a healthcare provider in analyzing patient reviews\nYour goal is to extract key information from the user message, including the patient\'s\nname, the doctor mentioned in the review, the review rating, a brief description of the\nreview, and whether the patient expressed satisfaction with their appointment. Go\nthrough the user\'s feedback step by step, and generate a structured output for further\nanalysis by the healthcare provider in the below format\n{\n"patient_name": <extract the patient’s first and last name from the corpus>,\n"consulting_doctor": <extract the doctor’s first and last name and credentials from\nthe\ncorpus>,\n"review_rating": <this has to be a number out of 5 points - if you cannot find a\nrating, output NULL>,\n"review_description": <summarize the review at most in 50 words>\n"satisfaction": <this has to be a TRUE or FALSE va

In [22]:
prompt.messages[1]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))

In [23]:
prompt.messages[1].prompt

PromptTemplate(input_variables=['question'], template='{question}')

In [ ]:
print(prompt.messages[1].prompt.template)

In [24]:
prompt.messages[1].prompt.template = 'Context information is below.\n---------------------\n{context}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nUse Bullet poits whenever possible in the answer.\nQuery: {query}\nAnswer: '

In [25]:
print(prompt.messages[1].prompt.template)

Context information is below.
---------------------
{context}
---------------------
Given the context information and not prior knowledge, answer the query.
Use Bullet poits whenever possible in the answer.
Query: {query}
Answer: 


Added the following line in the prompt: `Use Bullet poits whenever possible in the answer.`

In [29]:
handle = "pallaanji"
prompt_url = hub.push(f"{handle}/earnings-call-rag", prompt)

In [30]:
prompt_url

'https://smith.langchain.com/hub/pallaanji/earnings-call-rag/532d741a'

In [31]:
from langchain import hub
prompt = hub.pull("bhaskarjit/patient-review")
hub.push("pallaanji/patient-review", prompt)


'https://smith.langchain.com/hub/pallaanji/patient-review/81ce0445'